# Chest X-Ray Classification - Optimized for 80%+ Score

**Strategy:**
- Phase 1: Reproduce 80.122% baseline (ResNet18, simple config)
- Phase 2: Train additional models (ResNet34, EfficientNetV2-S)
- Phase 3: TTA + Ensemble for 85%+ target

**Baseline Model Config (80.122%):**
- Model: ResNet18 pretrained
- Image Size: 224px
- Batch Size: 32 (A100) or 12 (local)
- Epochs: 8-12
- Loss: CrossEntropy + Label Smoothing 0.05
- Optimizer: AdamW, LR 0.0003
- Scheduler: Cosine with 1 epoch warmup
- Weighted Sampler: True
- AMP: bfloat16

## 1. Environment Setup

In [ ]:
# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running on Google Colab")
except:
    IN_COLAB = False
    print("Running locally")

# Install dependencies
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q numpy pandas scikit-learn matplotlib tqdm pyyaml opencv-python seaborn albumentations

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os, sys, yaml, random

# Check GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    
# Enable TF32 for A100
torch.set_float32_matmul_precision('medium')
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
print(f"TF32 enabled: {torch.backends.cuda.matmul.allow_tf32}")

## 2. Data Upload & Setup

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Option 1: Clone from GitHub
    # !git clone YOUR_REPO_URL
    
    # Option 2: Upload zip file from Drive
    # !unzip "/content/drive/MyDrive/nycu-CSIC30014-LAB3.zip" -d /content/
    
    # Option 3: Manual upload (for testing)
    print("Please upload your project folder or use one of the options above")
    
    # Set working directory
    # os.chdir('/content/nycu-CSIC30014-LAB3')
else:
    # Local path
    os.chdir('C:/Users/thc1006/Desktop/114-1/nycu-CSIC30014-LAB3')

print(f"Current directory: {os.getcwd()}")
print(f"Files: {os.listdir('.')}")

## 3. Data Exploration

In [ ]:
# Load data
train_df = pd.read_csv('data/train_data.csv')
val_df = pd.read_csv('data/val_data.csv')
test_df = pd.read_csv('data/test_data.csv')

print(f"Train: {len(train_df)} samples")
print(f"Val: {len(val_df)} samples")
print(f"Test: {len(test_df)} samples")

# Class distribution
classes = ['normal', 'bacteria', 'virus', 'COVID-19']
train_dist = {cls: train_df[cls].sum() for cls in classes}
print("\nTrain distribution:")
for cls, count in train_dist.items():
    print(f"  {cls:10s}: {count:4d} ({count/len(train_df)*100:.1f}%)")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, (name, df) in zip(axes, [('Train', train_df), ('Val', val_df), ('Test', test_df)]):
    counts = [df[cls].sum() for cls in classes]
    ax.bar(classes, counts)
    ax.set_title(f"{name} Distribution (n={len(df)})")
    ax.set_ylabel('Count')
    ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

## 4. Core Training Functions

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from PIL import Image
from sklearn.metrics import f1_score, confusion_matrix
from tqdm.auto import tqdm
import math

# Dataset
class XRayDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = Path(img_dir)
        self.transform = transform
        self.classes = ['normal', 'bacteria', 'virus', 'COVID-19']
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / row['new_filename']
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
        
        # Get label (argmax of one-hot)
        label = np.argmax([row[cls] for cls in self.classes])
        
        return img, label, row['new_filename']

# Model builder
def build_model(name, num_classes=4):
    if name == "resnet18":
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif name == "resnet34":
        model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif name == "efficientnet_v2_s":
        model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError(f"Unknown model: {name}")
    return model

# Loss function with label smoothing
class LabelSmoothingCE(nn.Module):
    def __init__(self, smoothing=0.05, num_classes=4):
        super().__init__()
        self.smoothing = smoothing
        self.num_classes = num_classes
        
    def forward(self, logits, targets):
        log_probs = nn.functional.log_softmax(logits, dim=-1)
        targets_one_hot = nn.functional.one_hot(targets, self.num_classes).float()
        targets_smooth = (1 - self.smoothing) * targets_one_hot + self.smoothing / self.num_classes
        loss = -(targets_smooth * log_probs).sum(dim=-1).mean()
        return loss

# Cosine LR scheduler
def cosine_lr(optimizer, base_lr, warmup_steps, total_steps):
    def lr_lambda(step):
        if step < warmup_steps:
            return float(step) / float(max(1, warmup_steps))
        progress = float(step - warmup_steps) / float(max(1, total_steps - warmup_steps))
        return 0.5 * (1.0 + math.cos(math.pi * progress))
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# Training function
def train_epoch(model, loader, optimizer, scaler, device, loss_fn, use_amp=True):
    model.train()
    total_loss = 0
    all_preds, all_targets = [], []
    
    for imgs, targets, _ in tqdm(loader, desc="Training", leave=False):
        imgs, targets = imgs.to(device), targets.to(device)
        
        optimizer.zero_grad(set_to_none=True)
        
        if use_amp:
            with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
                logits = model(imgs)
                loss = loss_fn(logits, targets)
        else:
            logits = model(imgs)
            loss = loss_fn(logits, targets)
        
        if scaler and use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
        preds = logits.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    acc = np.mean(np.array(all_preds) == np.array(all_targets))
    f1 = f1_score(all_targets, all_preds, average='macro')
    return avg_loss, acc, f1

# Evaluation function
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    all_preds, all_targets = [], []
    
    for imgs, targets, _ in tqdm(loader, desc="Evaluating", leave=False):
        imgs, targets = imgs.to(device), targets.to(device)
        logits = model(imgs)
        preds = logits.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())
    
    acc = np.mean(np.array(all_preds) == np.array(all_targets))
    f1 = f1_score(all_targets, all_preds, average='macro')
    cm = confusion_matrix(all_targets, all_preds)
    return acc, f1, cm

print("Core functions defined successfully!")

## 5. Phase 1: Baseline Model (80.122% Reproduction)

In [ ]:
# Seed for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

# Config for baseline (80.122%)
BASELINE_CONFIG = {
    'model_name': 'resnet18',
    'img_size': 224,
    'batch_size': 32 if IN_COLAB else 12,  # A100 can handle 32
    'epochs': 12 if IN_COLAB else 8,
    'lr': 0.0003,
    'weight_decay': 0.0001,
    'label_smoothing': 0.05,
    'warmup_epochs': 1,
    'use_weighted_sampler': True,
    'num_workers': 4,
}

print("Baseline Configuration:")
for k, v in BASELINE_CONFIG.items():
    print(f"  {k:25s}: {v}")

In [ ]:
# Data transforms (BASELINE - keep it simple!)
train_transform = transforms.Compose([
    transforms.Resize((BASELINE_CONFIG['img_size'], BASELINE_CONFIG['img_size'])),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((BASELINE_CONFIG['img_size'], BASELINE_CONFIG['img_size'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = XRayDataset('data/train_data.csv', 'train_images', train_transform)
val_dataset = XRayDataset('data/val_data.csv', 'val_images', val_transform)

# Weighted sampler for class imbalance
if BASELINE_CONFIG['use_weighted_sampler']:
    train_df = pd.read_csv('data/train_data.csv')
    classes = ['normal', 'bacteria', 'virus', 'COVID-19']
    train_labels = np.argmax(train_df[classes].values, axis=1)
    class_counts = np.bincount(train_labels)
    class_weights = 1.0 / class_counts
    sample_weights = class_weights[train_labels]
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))
    shuffle = False
    print(f"Using WeightedRandomSampler | Class weights: {class_weights}")
else:
    sampler = None
    shuffle = True

# DataLoaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=BASELINE_CONFIG['batch_size'],
    sampler=sampler,
    shuffle=shuffle if sampler is None else False,
    num_workers=BASELINE_CONFIG['num_workers'],
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BASELINE_CONFIG['batch_size'],
    shuffle=False,
    num_workers=BASELINE_CONFIG['num_workers'],
    pin_memory=True
)

print(f"\nTrain batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

In [ ]:
# Build baseline model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model_baseline = build_model(BASELINE_CONFIG['model_name']).to(device)
model_baseline = model_baseline.to(memory_format=torch.channels_last)  # Optimize for A100

# Optimizer and scheduler
optimizer = optim.AdamW(
    model_baseline.parameters(),
    lr=BASELINE_CONFIG['lr'],
    weight_decay=BASELINE_CONFIG['weight_decay']
)

steps_per_epoch = len(train_loader)
total_steps = BASELINE_CONFIG['epochs'] * steps_per_epoch
warmup_steps = BASELINE_CONFIG['warmup_epochs'] * steps_per_epoch

scheduler = cosine_lr(optimizer, BASELINE_CONFIG['lr'], warmup_steps, total_steps)

# Loss function
loss_fn = LabelSmoothingCE(smoothing=BASELINE_CONFIG['label_smoothing'])

# AMP scaler
scaler = torch.cuda.amp.GradScaler() if device.type == 'cuda' else None

print(f"Model parameters: {sum(p.numel() for p in model_baseline.parameters()) / 1e6:.2f}M")
print(f"Total steps: {total_steps} | Warmup steps: {warmup_steps}")

In [ ]:
# Train baseline model
print("\n" + "="*60)
print("Starting Baseline Training (Target: 80%+ F1)")
print("="*60 + "\n")

best_val_f1 = 0
history = {'train_loss': [], 'train_acc': [], 'train_f1': [], 'val_acc': [], 'val_f1': []}

for epoch in range(BASELINE_CONFIG['epochs']):
    print(f"\nEpoch {epoch+1}/{BASELINE_CONFIG['epochs']}")
    
    # Train
    train_loss, train_acc, train_f1 = train_epoch(
        model_baseline, train_loader, optimizer, scaler, device, loss_fn, use_amp=True
    )
    
    # Validate
    val_acc, val_f1, val_cm = evaluate(model_baseline, val_loader, device)
    
    # Update scheduler
    for _ in range(steps_per_epoch):
        scheduler.step()
    
    # Log
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['train_f1'].append(train_f1)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    
    print(f"  Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | F1: {train_f1:.4f}")
    print(f"  Val   Acc: {val_acc:.4f} | F1: {val_f1:.4f}")
    
    # Save best model
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save({
            'epoch': epoch,
            'model_state_dict': model_baseline.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_f1': val_f1,
            'config': BASELINE_CONFIG
        }, 'baseline_best.pt')
        print(f"  -> Saved best model (Val F1: {val_f1:.4f})")

print(f"\n{'='*60}")
print(f"Baseline Training Complete!")
print(f"Best Val F1: {best_val_f1:.4f} (Target: 0.80+)")
print(f"{'='*60}")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# F1 Score
axes[1].plot(history['train_f1'], label='Train F1', marker='o')
axes[1].plot(history['val_f1'], label='Val F1', marker='s')
axes[1].axhline(y=0.80, color='r', linestyle='--', label='Target (80%)')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Macro F1 Score')
axes[1].set_title(f'F1 Score (Best Val: {best_val_f1:.4f})')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('baseline_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

# Confusion matrix
classes = ['Normal', 'Bacteria', 'Virus', 'COVID-19']
plt.figure(figsize=(8, 6))
sns.heatmap(val_cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.title(f'Validation Confusion Matrix (F1: {val_f1:.4f})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('baseline_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Generate Baseline Submission

In [ ]:
# Load best baseline model
checkpoint = torch.load('baseline_best.pt')
model_baseline.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded best baseline model (Val F1: {checkpoint['val_f1']:.4f})")

# Prepare test data
test_dataset = XRayDataset('data/test_data.csv', 'test_images', val_transform)
test_loader = DataLoader(
    test_dataset,
    batch_size=BASELINE_CONFIG['batch_size'],
    shuffle=False,
    num_workers=BASELINE_CONFIG['num_workers'],
    pin_memory=True
)

# Generate predictions
@torch.no_grad()
def predict(model, loader, device):
    model.eval()
    all_probs = []
    all_filenames = []
    
    for imgs, _, filenames in tqdm(loader, desc="Predicting"):
        imgs = imgs.to(device)
        logits = model(imgs)
        probs = torch.softmax(logits, dim=1)
        all_probs.append(probs.cpu().numpy())
        all_filenames.extend(filenames)
    
    all_probs = np.vstack(all_probs)
    return all_probs, all_filenames

baseline_probs, filenames = predict(model_baseline, test_loader, device)

# Create submission
submission_df = pd.DataFrame({
    'new_filename': filenames,
    'normal': baseline_probs[:, 0],
    'bacteria': baseline_probs[:, 1],
    'virus': baseline_probs[:, 2],
    'COVID-19': baseline_probs[:, 3]
})

# Convert to one-hot (hardmax)
pred_classes = baseline_probs.argmax(axis=1)
one_hot = np.eye(4)[pred_classes]
submission_df[['normal', 'bacteria', 'virus', 'COVID-19']] = one_hot

# Save
submission_df.to_csv('submission_baseline.csv', index=False)
print(f"\nBaseline submission saved to: submission_baseline.csv")
print(f"Predictions: {len(submission_df)}")
print(f"Class distribution:")
for i, cls in enumerate(['normal', 'bacteria', 'virus', 'COVID-19']):
    print(f"  {cls:10s}: {(pred_classes == i).sum():4d} ({(pred_classes == i).sum()/len(pred_classes)*100:.1f}%)")

## 7. Phase 2: Additional Models (Optional - for Ensemble)

In [ ]:
# Train additional models for ensemble
# This is optional - only if you have time and want to push beyond 80%

ADDITIONAL_MODELS = [
    {'name': 'resnet34', 'img_size': 256, 'epochs': 10},
    {'name': 'efficientnet_v2_s', 'img_size': 288, 'epochs': 12},
]

# Uncomment to train additional models
# trained_models = []
# for config in ADDITIONAL_MODELS:
#     print(f"\nTraining {config['name']}...")
#     # Train model (similar to baseline)
#     # Save model and predictions
#     pass

## 8. Phase 3: Test-Time Augmentation (TTA)

In [ ]:
# TTA - Multiple augmented predictions averaged
print("\n" + "="*60)
print("Applying Test-Time Augmentation (TTA)")
print("="*60 + "\n")

# Define TTA transforms
tta_transforms = [
    # Original
    transforms.Compose([
        transforms.Resize((BASELINE_CONFIG['img_size'], BASELINE_CONFIG['img_size'])),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Horizontal flip
    transforms.Compose([
        transforms.Resize((BASELINE_CONFIG['img_size'], BASELINE_CONFIG['img_size'])),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Slight rotation +5
    transforms.Compose([
        transforms.Resize((BASELINE_CONFIG['img_size'], BASELINE_CONFIG['img_size'])),
        transforms.RandomRotation(degrees=(5, 5)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Slight rotation -5
    transforms.Compose([
        transforms.Resize((BASELINE_CONFIG['img_size'], BASELINE_CONFIG['img_size'])),
        transforms.RandomRotation(degrees=(-5, -5)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
]

@torch.no_grad()
def predict_tta(model, test_df, img_dir, transforms_list, device, batch_size=32):
    model.eval()
    all_probs_tta = []
    
    for i, transform in enumerate(transforms_list):
        print(f"TTA {i+1}/{len(transforms_list)}...", end=' ')
        dataset = XRayDataset(test_df, img_dir, transform)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
        
        batch_probs = []
        for imgs, _, _ in loader:
            imgs = imgs.to(device)
            logits = model(imgs)
            probs = torch.softmax(logits, dim=1)
            batch_probs.append(probs.cpu().numpy())
        
        all_probs_tta.append(np.vstack(batch_probs))
        print(f"Done")
    
    # Average all TTA predictions
    avg_probs = np.mean(all_probs_tta, axis=0)
    return avg_probs

# Apply TTA
tta_probs = predict_tta(
    model_baseline, 
    'data/test_data.csv',
    'test_images',
    tta_transforms,
    device,
    batch_size=BASELINE_CONFIG['batch_size']
)

# Create TTA submission
tta_pred_classes = tta_probs.argmax(axis=1)
tta_one_hot = np.eye(4)[tta_pred_classes]

submission_tta = pd.DataFrame({
    'new_filename': filenames,
    'normal': tta_one_hot[:, 0],
    'bacteria': tta_one_hot[:, 1],
    'virus': tta_one_hot[:, 2],
    'COVID-19': tta_one_hot[:, 3]
})

submission_tta.to_csv('submission_baseline_tta.csv', index=False)
print(f"\nTTA submission saved to: submission_baseline_tta.csv")
print(f"\nTTA Class distribution:")
for i, cls in enumerate(['normal', 'bacteria', 'virus', 'COVID-19']):
    print(f"  {cls:10s}: {(tta_pred_classes == i).sum():4d} ({(tta_pred_classes == i).sum()/len(tta_pred_classes)*100:.1f}%)")

# Compare with baseline
differences = (pred_classes != tta_pred_classes).sum()
print(f"\nDifferences from baseline: {differences}/{len(pred_classes)} ({differences/len(pred_classes)*100:.1f}%)")
print(f"Expected improvement: +0.5% to +1.5% F1 score")

## 9. Download Results

In [ ]:
if IN_COLAB:
    from google.colab import files
    
    print("Downloading submission files...")
    files.download('submission_baseline.csv')
    files.download('submission_baseline_tta.csv')
    files.download('baseline_best.pt')
    files.download('baseline_training_history.png')
    files.download('baseline_confusion_matrix.png')
    print("Download complete!")
else:
    print("Files saved locally:")
    print("  - submission_baseline.csv")
    print("  - submission_baseline_tta.csv")
    print("  - baseline_best.pt")
    print("  - baseline_training_history.png")
    print("  - baseline_confusion_matrix.png")

## 10. Summary & Next Steps

In [ ]:
print("\n" + "="*70)
print(" "*25 + "FINAL SUMMARY")
print("="*70)
print(f"\n1. BASELINE MODEL (ResNet18):")
print(f"   - Val F1 Score: {best_val_f1:.4f}")
print(f"   - Target: 0.80+ (80.122% baseline)")
print(f"   - Status: {'PASSED' if best_val_f1 >= 0.80 else 'NEEDS MORE TRAINING'}")
print(f"\n2. SUBMISSIONS GENERATED:")
print(f"   - submission_baseline.csv (standard prediction)")
print(f"   - submission_baseline_tta.csv (with TTA, expected +0.5-1.5% improvement)")
print(f"\n3. RECOMMENDED SUBMISSION:")
print(f"   - Use: submission_baseline_tta.csv")
print(f"   - Expected Public Score: 80-82%")
print(f"\n4. FURTHER IMPROVEMENTS (Optional):")
print(f"   - Train ResNet34 or EfficientNetV2-S (Section 7)")
print(f"   - Ensemble multiple models (soft voting)")
print(f"   - Longer training (20+ epochs with early stopping)")
print(f"   - Advanced data augmentation (CLAHE, MixUp)")
print(f"   - Pseudo-labeling with test set")
print("\n" + "="*70)
print("\nGood luck with your submission!")
print("="*70)